In [1]:
import os
from dotenv import load_dotenv

# Load all environment variables from .env file
load_dotenv()

## LLM
openai_api_key = os.getenv('OPENAI_API_KEY')

## Pinecone Vector Database
pinecone_api_key = os.getenv('PINECONE_API_KEY')


In [2]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)



/Users/hrithikkoduri/Desktop/Course/venv/lib/python3.13/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
import time

index_name = "rag-multi-query-index" # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [4]:
# Load blog
import bs4
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone
from pprint import pprint

#### INDEXING ####

# Load Document (Uploading one file at a time)
pdf_file_path = "./data/langchain_turing.pdf"
loader = PyPDFLoader(pdf_file_path)

docs = loader.load()

# Upload muiltiple PDF files from a directory
# pdf_file_paths = <enter your path here>
# loader = PyPDFDirectoryLoader(pdf_file_paths)

# docs_dir = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, 
    chunk_overlap=500)

# Make splits
splits = text_splitter.split_documents(docs)

# Index
vectorstore = Pinecone.from_documents(
    documents=splits, 
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"), 
    index_name=index_name
)


In [5]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 5, "score_threshold": 0.5},
)

## Multi Query RAG

Python doc- https://python.langchain.com/docs/how_to/MultiQueryRetriever/

![Multi Query RAG](./images/multi_query_rag.png)

In [22]:
question = "How does LangChain leverage modular components like LangGraph, LangSmith, and LangServe to address challenges in building scalable and secure LLM-powered applications?"


In [21]:
from pydantic import BaseModel

class Question(BaseModel):
    generated_questions: list[str]



In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


generated_questions_prompt = prompt_perspectives.invoke(
    {"question": question}
)
llm_with_structured_output = llm.with_structured_output(Question)


generated_questions = llm_with_structured_output.invoke(generated_questions_prompt)

print(generated_questions.generated_questions)

['What are the roles of LangGraph, LangSmith, and LangServe in LangChain for developing scalable and secure applications with LLMs?', 'In what ways does LangChain utilize modular components such as LangGraph, LangSmith, and LangServe to enhance the scalability and security of LLM applications?', 'Can you explain how LangChain incorporates LangGraph, LangSmith, and LangServe to tackle issues related to scalability and security in LLM-powered applications?', 'How do the modular components of LangChain, including LangGraph, LangSmith, and LangServe, contribute to building secure and scalable applications using LLM technology?', 'What challenges in LLM application development does LangChain address through its use of components like LangGraph, LangSmith, and LangServe?']


In [23]:
for que in generated_questions.generated_questions:
    print(que)

What are the roles of LangGraph, LangSmith, and LangServe in LangChain for developing scalable and secure applications with LLMs?
In what ways does LangChain utilize modular components such as LangGraph, LangSmith, and LangServe to enhance the scalability and security of LLM applications?
Can you explain how LangChain incorporates LangGraph, LangSmith, and LangServe to tackle issues related to scalability and security in LLM-powered applications?
How do the modular components of LangChain, including LangGraph, LangSmith, and LangServe, contribute to building secure and scalable applications using LLM technology?
What challenges in LLM application development does LangChain address through its use of components like LangGraph, LangSmith, and LangServe?


In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def generate_questions(question):   

    # Multi Query: Different Perspectives
    template = """You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines. Original question: {question}"""
    prompt_perspectives = ChatPromptTemplate.from_template(template)


    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


    generated_questions_prompt = prompt_perspectives.invoke(
        {"question": question}
    )
    llm_with_structured_output = llm.with_structured_output(Question)


    generated_questions = llm_with_structured_output.invoke(generated_questions_prompt)

    return generated_questions.generated_questions


In [13]:
all_docs_retriever = generate_questions | retriever.map()

question = "How does LangChain leverage modular components like LangGraph, LangSmith, and LangServe to address challenges in building scalable and secure LLM-powered applications?"

all_docs = all_docs_retriever.invoke(question)



In [14]:
for i,  docs in enumerate(all_docs):
    print(f"***********Results from {i+1}th query:***********")
    for doc in docs:
        print(doc.page_content[:100])
        print("-"*100)

***********Results from 1th query:***********
10 Vasilios Mavroudis
3.4 Integration with LangChain and LangSmith
LangGraph integrates seamlessly w
----------------------------------------------------------------------------------------------------
2 Vasilios Mavroudis
stateful, and contextually aware applications with ease. Its suite of compo-
ne
----------------------------------------------------------------------------------------------------
LangChain
Vasilios Mavroudis
Alan Turing Institute
vmavroudis@turing.ac.uk
Abstract. LangChain is a 
----------------------------------------------------------------------------------------------------
6 Vasilios Mavroudis
1.5 Advanced Components
Beyond these core elements, LangChain offers advanced m
----------------------------------------------------------------------------------------------------
LangChain 3
needs, providing a flexible foundation for building scalable, secure, and multi-
functio
---------------------------------------------

In [ ]:
def get_unique_docs(all_docs):
    unique_docs = []
    for docs in all_docs:
        for doc in docs:
            if doc not in unique_docs:
                unique_docs.append(doc)
    return unique_docs

unique_docs = get_unique_docs(all_docs)

len(unique_docs)


7

In [16]:
final_retrieval_chain = generate_questions | retriever.map() | get_unique_docs

question = "How does LangChain leverage modular components like LangGraph, LangSmith, and LangServe to address challenges in building scalable and secure LLM-powered applications?"

context = final_retrieval_chain.invoke(question)

len(context)


7

In [17]:
from langchain_core.messages import HumanMessage, SystemMessage

def generate_response(question):
    rag_system_prompt = """
        You are an AI language model assistant. Your task is to answer the user question based on the provided context.

        Context:
        {context}
    """

    rag_user_prompt = """
        Question: {question}
    """
    context = final_retrieval_chain.invoke(question)

    rag_system_message = SystemMessage(content=rag_system_prompt.format(context=context))
    rag_user_message = HumanMessage(content=rag_user_prompt.format(question=question))

    rag_prompt = [rag_system_message , rag_user_message]

    response = llm.invoke(rag_prompt)
    
    return response

In [ ]:
question = "How does LangChain leverage modular components like LangGraph, LangSmith, and LangServe to address challenges in building scalable and secure LLM-powered applications?"

In [18]:
response = generate_response(question)

In [19]:
from IPython.display import Markdown

Markdown(response.content)

LangChain leverages its modular components—LangGraph, LangSmith, and LangServe—to effectively address the challenges of building scalable and secure applications powered by large language models (LLMs) in several ways:

1. **LangGraph for Stateful Process Modeling**: 
   - LangGraph allows developers to structure applications using nodes and edges, facilitating complex branching and multi-agent workflows. This modular approach enables the creation of stateful and contextually aware applications, which can manage intricate processes and interactions efficiently. By modeling workflows as graphs, developers can visualize and optimize the flow of data and control, enhancing scalability.

2. **LangSmith for Monitoring and Evaluation**: 
   - LangSmith provides essential tools for real-time performance monitoring, error tracking, and version control. It addresses key challenges in observability and optimization by allowing developers to trace interactions with LLMs and external data sources. This capability is crucial for identifying bottlenecks and ensuring that applications maintain high performance and reliability over time. LangSmith's evaluation tools also enable developers to test applications under real-world conditions, ensuring they meet performance standards.

3. **LangServe for Scalable API Deployment**: 
   - LangServe simplifies the process of deploying LangChain applications as scalable REST APIs. It supports load balancing and can handle multiple API requests simultaneously, which is essential for applications experiencing high traffic. LangServe also includes features for auto-scaling, allowing the number of instances to adjust dynamically based on demand. This ensures consistent performance and responsiveness, making it suitable for production environments.

Together, these components create a comprehensive toolkit that not only simplifies the development lifecycle of LLM applications but also enhances their scalability and security. By modularizing functionalities, LangChain allows developers to customize and extend their applications while addressing the complexities associated with integrating LLMs, managing state, and ensuring secure interactions with external systems. This modular architecture ultimately empowers developers to build innovative, efficient, and secure LLM-powered applications across various domains.